# IMD Bet Game

In the IMD bet game, your next move depends on the number of eyes you throw with the dice.


## Rules

**Input:**

* Floor 'f', where step = max(0,f)
* Epoch = 100
* Seed
* Number of rounds = 500

**For each round:**

* For each epoch (roll)
    * Roll the dice
    * Determine the next step
        * Case 1, 2: decrease step by one
        * Case 3, 4, 5: increase step by one
        * Case 6
            * Roll the dice
            * Move according to result of dice
        * There is a 0,1% probability that assign the step to 0 (zero) in each step.
        * Store the current step
* Round[i] = steps

## My Solution

I modeled that problem in OOP with classes. I created four classes:

* Dice
* Elevator
* Simulator
* Writer
    
The module *betGame* runs the program.

### Dice

It represents a traditional dice with six different faces. Each face has an integer between 1 and 6. The throwing is done by simply generating a random integer from 1 to 6. I did not use a seed because *randint* has worked well without a seed.

In [16]:
from random import randint

class Dice(object):
    
    def __init__(self):
        self.__MIN = 1
        self.__MAX = 6
        
    '''
    The act of throw a dice is done by simply generating a random integer from 1 to 6
    '''
    def throw(self):
        return randint(self.__MIN, self.__MAX)

### Elevator

It simulates an elevator which starts at some floor, cannot go below zero, and should not have a maximum floor.
It starts at the floor zero.

In [21]:
class Elevator(object):

    def __init__(self, current_floor = 0):
        '''
        Constructor that receives a current_floor and defines the minimum floor
        '''
        self.__MIN = 0
        self.__current_floor = current_floor
    
    '''
    It simulates the elevator going up x floors
    '''
    def move_up(self, x):
        if(x > 0):
            self.__current_floor += x
        else:
            raise ValueError('Cannot move up with the value: ' + str(x))
    
    '''
    It simulates the elevator going down x floors
    '''
    def move_down(self, x = 1):
        if(self.__current_floor - x >= 0 and x > 0): # Do not go below zero
            self.__current_floor -= x
        else:
            raise ValueError('Cannot move down with the value: ' + str(x))
    
    
    def get_current_floor(self):
        return self.__current_floor

    def set_current_floor(self, value):
            self.__current_floor = value
            
    current_floor = property(get_current_floor, set_current_floor, None, None)

### Simulator

It simulates the Bet Game with 500 round, each one with 100 steps. I used a numpy array and followed the official documentation and a recomendation (http://stackoverflow.com/questions/568962/how-do-i-create-an-empty-array-matrix-in-numpy). I used randint without a seed because it worked well.

In [18]:
from random import randint
import numpy

class Simulator(object):

    def __init__(self):
        '''
        Initializes the threshold variables and the array which will store the
        result
        '''
        self.__ROUNDS = 500
        self.__EPOCH = 100
        self.__result = numpy.zeros(shape = (self.__ROUNDS, self.__EPOCH), dtype = numpy.int)
        
    '''
    Runs simulation
    '''
    def run(self):
        dice = Dice()
        for i in range(self.__ROUNDS):
            elevator = Elevator()
            for j in range(self.__EPOCH):
                '''
                There is a 0,1% probability that assign the step to 0 (zero)
                in each step.
                '''
                if randint(1, 1000) == 1:
                    elevator.current_floor = 0
                else:
                    dots = dice.throw()
                    try:
                        if dots == 1 or dots == 2:
                            elevator.move_down() # move one floor down
                        elif dots > 2 and dots < 6:
                            elevator.move_up(1) # move one floor up
                        else:
                            x = dice.throw()
                            elevator.move_up(x) # move 1..6 floors up
                    except ValueError:
                        pass
                numpy.put(self.__result[i], j, elevator.current_floor)
                    
                    
    def get_result(self):
        return self.__result
    
    def get_rounds(self):
        return self.__ROUNDS

    def get_epoch(self):
        return self.__EPOCH
    
    result = property(get_result, None, None, None)
    ROUNDS = property(get_rounds, None, None, None)
    EPOCH = property(get_epoch, None, None, None)

### Writer

Class for writing the simulation result to a file.

In [19]:
class WriteToFile(object):

    def __init__(self, path):
        self.__path = path
        
    '''
    Write the simulation result to a file
    '''
    def write(self, result, rows, colums):
        res_file = open(self.__path, 'w')
        for i in range(rows):
            for j in range(colums):
                res_file.write(str(result[i][j]) + ' ')
            res_file.write('\n')

### Main

Here I run the program. Firstly, I run the simulation and after that I write the result (all movements stored in the array) to a file named *result.out*.

In [20]:
print('Simulating...')
simulator = Simulator()
simulator.run()

print('\nWriting...')
writer = WriteToFile('result.out')
writer.write(simulator.result, simulator.ROUNDS, simulator.EPOCH)

print('\n-- END --')

Simulating...

Writing...

-- END --





**Rubem Kalebe (rubemkalebe@gmail.com)**